In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
from tqdm.notebook import tqdm
%matplotlib inline

import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


In [3]:
## Load the dateset
TRAIN_DIR = './FER2013/train/'
TEST_DIR = './FER2013/test/'

In [4]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(    
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.3,
)

validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all train images
train_generator = train_data_gen.flow_from_directory(
        TRAIN_DIR,
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=True)

# Preprocess all validation images
validation_generator = validation_data_gen.flow_from_directory(
        TEST_DIR,
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=True)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
# train_X = np.concatenate([train_generator.next()[0] for i in range(train_generator.__len__())])
# train_Y = np.concatenate([train_generator.next()[1] for i in range(train_generator.__len__())])
# val_X = np.concatenate([validation_generator.next()[0] for i in range(validation_generator.__len__())])
# val_Y = np.concatenate([validation_generator.next()[1] for i in range(validation_generator.__len__())])

In [18]:
train_X = np.empty([28709, 48, 48, 1])
train_Y = np.empty([28709, 7])
start = 0
end = 0
for i in range(0, 449):
    image, label = train_generator.next()
    end = end + image.shape[0]
    train_X[start:end, :, :, :] = image
    train_Y[start:end, :] = label
    start = start + image.shape[0]

In [16]:
val_X = np.empty([7178, 48, 48, 1])
val_Y = np.empty([7178, 7])
start = 0
end = 0
for i in range(0, 113):
    image, label = validation_generator.next()
    end = end + image.shape[0]
    val_X[start:end, :, :, :] = image
    val_Y[start:end, :] = label
    start = start + image.shape[0]

### CNN: Model 1

In [ ]:
# create model structure
simple_model = Sequential([
    #block-1
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    
    #block-2
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    
    #block-3
    Dense(7, activation='softmax')
])

simple_model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001,epsilon=1e-06),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

simple_model.summary()
# Train the neural network/model
simple_model_info = simple_model.fit(
        train_generator,
        epochs=10,
        validation_data=validation_generator)

In [ ]:
# plot accuracy
plt.plot(simple_model_info.history['accuracy'])
plt.plot(simple_model_info.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# plot loss
plt.plot(simple_model_info.history['loss'])
plt.plot(simple_model_info.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

### CNN: Model 2

In [ ]:
# create model structure
emotion_model = Sequential([
    #block-1
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)),
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    #block-2
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    #block-3
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    
    #block-4
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    
    #block-5
    Dense(512, activation='relu'),   
    Dropout(0.2),
    Dense(7, activation='softmax')
])

emotion_model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001,epsilon=1e-06),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

emotion_model.summary()

# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=60,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

In [ ]:
# plot accuracy
plt.plot(emotion_model_info.history['accuracy'])
plt.plot(emotion_model_info.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# plot loss
plt.plot(emotion_model_info.history['loss'])
plt.plot(emotion_model_info.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
np.random.seed(100)
images, labels = next(validation_generator)
test_loss, test_acc = emotion_model.evaluate(images, labels, verbose=2)

In [ ]:
emotion_model.save('my_cnn_model.h5')